# Difference bases

In [ ]:
#@title Data and verification

import numpy as np

# --- Leech-like Combination ---
def combine_A_B(A_set, B_set, m_modulus_for_B):
  """Combines A and B using L = {a*m + b} and finds empirical_n.

  m_modulus_for_B is the 'm' used when constructing B.
  """
  print(f"\nCombining A={A_set} and B={B_set} with m(N)={m_modulus_for_B}")
  L_set = set()
  for a_val in A_set:
    for b_val in B_set:
      L_set.add(a_val * m_modulus_for_B + b_val)

  if not L_set:
    return []
  sorted_L = sorted(list(L_set))
  return sorted_L


def get_score(combined_set):
  mylist = list(combined_set)

  temp_B_start_list = []
  for x in mylist:  # Ensure all elements are valid integers
    if isinstance(x, (int, float, np.integer, np.floating)) and not np.isnan(x):
      temp_B_start_list.append(int(x))
    else:
      return -float('inf'), False  # Invalid element in list

  B_start = sorted(list(set(temp_B_start_list)))
  if (
      0 not in B_start
  ):  # Ensure 0 is in B_start for the greedy completion strategy
    B_start.append(0)

  differences = set()
  # print(len(B_start))
  if len(B_start) > 2000:
    return -float('inf')
  for i in range(len(B_start)):
    for j in range(i + 1, len(B_start)):
      diff = B_start[j] - B_start[i]
      differences.add(diff)

  max_achieved_diff = 0
  if differences:
    max_achieved_diff = max(differences)

  for v_val in range(1, max_achieved_diff + 2):
    if v_val not in differences:
      if v_val == 1:
        return -100
      # print(v_val,[int(x) for x in combined_set], combined_set, differences)
      return len(B_start) ** 2 / (v_val - 1)


A=[0, 1, 4, 6]
B=[0, 1, 70, 83, 255, 297, 384, 391, 550, 555, 647, 656, 710, 996, 1020, 1232, 1257, 1272, 1452, 1456, 1536, 1614, 1745, 1765, 1948, 2047, 2150, 2188, 2214, 2395, 2407, 2585, 2612, 2628, 2739, 2758, 2858, 2902, 2974, 3006, 3027, 3245, 3392, 3477, 3526, 3615, 3675, 3727, 3849, 3906, 3935, 4043, 4049, 4253, 4410, 4445, 4578, 4580, 4821, 4855, 4911, 4934, 4973, 5032, 5099, 5149, 5160, 5411, 5452, 5518, 5526, 5658, 5833, 5855, 5926, 5943, 5957, 5994, 6139, 6185, 6281, 6592, 6622, 6669, 6687, 6697, 6742, 6745, 6778, 6967]
L = combine_A_B(A, B, 89**2 + 89 + 1)
get_score(L)


**Prompt used**

Problem: Finding Optimal Difference Bases for Integer Intervals

This task is about finding a small set of non-negative integers $B$ such that its set of differences $B-B = (b_1 - b_2 \mid b_1, b_2 \in B )$ covers all integers in the interval $[0, k]$ for a large integer $k \ge 1$. Such a set $B$ is called a difference basis for $[0, k]$. Let $\Delta[k]$ denote the size of the smallest such set $B$. We are interested in the quantity $\eth[k]^2 = (\Delta[k])^2 / k$. The goal is to find a set $B$ that leads to a small value of $\eth[k]^2$. This is equivalent to maximizing the score $-(\Delta[k])^2 / k$.

Your task is to implement a Python function `search_for_best_basis() -> List[int]`.
This function should search for a list of candidate integers `B_candidate_list`.
The elements of `B_candidate_list` should be non-negative integers.

The quality of `B_candidate_list` is determined by the following scoring mechanism, implemented in a function `get_score(B_candidate_list: List[int]) -> float` which you will have access to:

```
def get_score(combined_set):
    differences = set()
    for i in range(len(combined_set)):
        for j in range(i + 1, len(combined_set)):
            diff = combined_set[j] - combined_set[i]
            differences.add(diff)
    max_achieved_diff = 0
    if differences:
        max_achieved_diff = max(differences)
    for v_val in range(1, max_achieved_diff + 2):
        if v_val not in differences:
            #print("First integer NOT representable as a difference: v_val")
            if v_val == 1:
                return -100
            return -len(combined_set)**2 / (v_val - 1)
```

You want the score returned by `get_score` to be as high (i.e., as close to 0) as possible.

You should try ideas such as Singer difference sets, Leech and Golay codes, etc.

One good way to construct such sets is to combine difference bases (sets such that the differences contain every number from 0 to some n) with cyclic difference sets (sets such that the differences contain every number modulo m for some m). One can combine such sets with the Leech construction:

```
def combine_A_B(A_set, B_set, m_modulus_for_B):
    """
    Combines A and B using L = [a*m + b].
    m_modulus_for_B is the 'm' used when constructing B.
    """
    L_set = set()
    for a_val in A_set:
        for b_val in B_set:
            L_set.add(a_val * m_modulus_for_B + b_val)
    sorted_L = sorted(list(L_set))
    return sorted_L
```

This is typically going to result in a good construction.

Your task is to write a search function called search_for_best_basis() that runs for at most 20000 seconds, and then returns the best construction it has found. The search function can evaluate as many constructions it wants (with the globally available get_score() function) within the 20000s time frame.

You can access the best construction we have found so far through the best_B_prev global variable.


In [ ]:
#@title Initial program (written mostly by Gemini)

import itertools
import logging
import time
from scipy import integrate
import numpy as np
from scipy import optimize
import warnings
import random
import re
from typing import Any, List, Tuple
from collections.abc import Callable, Mapping, Set
import scipy.linalg as la
import collections
import copy
import math
import numba

njit = numba.njit

def greedy_difference_basis(target_coverage_A, max_elements_A, max_val_A):
  """Greedily constructs a difference basis A trying to cover 1 to target_coverage_A.

  Stops if target_coverage_A is met, max_elements_A is reached, or no good next
  element is found.
  """
  A = {0}  # Start with 0 to make differences easier
  differences_A = {0}  # To store |a_i - a_j|

  current_max_covered_A = 0

  # print(f"Greedy A: Target coverage {target_coverage_A}, max elements {max_elements_A}, max val {max_val_A}")

  for _ in range(max_elements_A - 1):  # We already have one element (0)
    best_next_a = -1
    best_new_coverage_extension = (
        current_max_covered_A  # How far does it extend contiguous coverage
    )

    # Try candidate next elements
    # Candidates could be random or sequential
    for candidate_a in range(1, max_val_A + 1):
      if candidate_a in A:
        continue

      # Calculate new differences if candidate_a is added
      temp_new_diffs = set()
      for existing_a in A:
        temp_new_diffs.add(abs(candidate_a - existing_a))

      # Check how far contiguous coverage extends
      temp_all_diffs = differences_A.union(temp_new_diffs)

      temp_max_contiguously_covered = 0
      for i in range(
          1, len(temp_all_diffs) + target_coverage_A + 2
      ):  # Check a bit beyond
        if i not in temp_all_diffs:
          temp_max_contiguously_covered = i - 1
          break
      else:  # All checked were covered
        temp_max_contiguously_covered = (
            max(temp_all_diffs) if temp_all_diffs else 0
        )

      if temp_max_contiguously_covered > best_new_coverage_extension:
        best_new_coverage_extension = temp_max_contiguously_covered
        best_next_a = candidate_a

      if temp_max_contiguously_covered == best_new_coverage_extension:
        if np.random.rand() < 0.5:  # Randomly break ties
          best_next_a = candidate_a

    if best_next_a != -1:
      # print(f"  Greedy A: Adding {best_next_a}. New contiguous coverage extends to {best_new_coverage_extension}")
      A.add(best_next_a)
      new_diffs_with_best_a = set()
      for existing_a in A - {
          best_next_a
      }:  # Differences with elements already in A before adding best_next_a
        new_diffs_with_best_a.add(abs(best_next_a - existing_a))
      differences_A.update(new_diffs_with_best_a)
      current_max_covered_A = best_new_coverage_extension
      if current_max_covered_A >= target_coverage_A:
        # print(f"  Greedy A: Reached target coverage {target_coverage_A}.")
        break
    else:
      # print("  Greedy A: No element found to extend contiguous coverage significantly.")
      break

  # print(f"Greedy A: Final set A = {sorted(list(A))}, covers 1 to {current_max_covered_A}")
  return sorted(list(A))


# --- Greedy Algorithm for "Covering" Difference Set B (mod m) ---
def greedy_covering_set_B(m_modulus, max_elements_B):
  """Greedily constructs a set B such that {b_i - b_j mod m} covers all non-zero residues mod m."""
  B = {0}  # Start with 0
  covered_residues_B = {0}  # Stores { (b_i - b_j) mod m }

  # print(f"Greedy B: Modulus m={m_modulus}, max elements {max_elements_B}")

  for _ in range(max_elements_B - 1):  # We already have one element (0)
    if len(covered_residues_B) == m_modulus:  # All residues (0 to m-1) covered
      # print("  Greedy B: All residues modulo m are covered.")
      break

    best_next_b = -1
    max_newly_covered_count = -1

    # Try candidate next elements (0 to m-1)
    # To make it somewhat random but systematic, shuffle candidates
    candidates_b = list(range(m_modulus))
    random.shuffle(candidates_b)

    for candidate_b in candidates_b:
      if candidate_b in B:
        continue

      # Calculate new residues if candidate_b is added
      temp_newly_covered_by_candidate = set()
      for existing_b in B:
        temp_newly_covered_by_candidate.add(
            (candidate_b - existing_b + m_modulus) % m_modulus
        )
        temp_newly_covered_by_candidate.add(
            (existing_b - candidate_b + m_modulus) % m_modulus
        )

      # Count how many *new* residues this candidate_b would cover
      # (residues not already in covered_residues_B)
      count_of_actually_new = len(
          temp_newly_covered_by_candidate - covered_residues_B
      )

      if count_of_actually_new > max_newly_covered_count:
        max_newly_covered_count = count_of_actually_new
        best_next_b = candidate_b

      if count_of_actually_new == max_newly_covered_count:
        if np.random.rand() < 0.5:  # Randomly break ties
          best_next_b = candidate_b

    if (
        best_next_b != -1 and max_newly_covered_count > 0
    ):  # Add if it covers at least one new residue
      # print(f"  Greedy B: Adding {best_next_b}. Covers {max_newly_covered_count} new residue(s).")
      B.add(best_next_b)
      # Update all covered residues with the new element
      new_diffs_with_best_b = set()
      for existing_b in B - {best_next_b}:
        new_diffs_with_best_b.add(
            (best_next_b - existing_b + m_modulus) % m_modulus
        )
        new_diffs_with_best_b.add(
            (existing_b - best_next_b + m_modulus) % m_modulus
        )
      new_diffs_with_best_b.add(0)  # Difference with itself
      covered_residues_B.update(new_diffs_with_best_b)
    else:
      # print("  Greedy B: No element found to cover significantly more new residues, or all covered.")
      break

  all_target_residues = set(range(m_modulus))
  uncovered = all_target_residues - covered_residues_B
  if uncovered:
    print(
        '  Greedy B: Warning! Failed to cover all residues. Uncovered:'
        f' {uncovered}'
    )
  else:
    # print(f"  Greedy B: Successfully covered all residues 0 to {m_modulus-1}.")
    pass

  # print(f"Greedy B: Final set B = {sorted(list(B))}, covers {len(covered_residues_B)}/{m_modulus} residues mod {m_modulus}")
  return sorted(list(B))


# --- Polynomial and Number Theoretic Helpers ---
njit = numba.njit

# --- High-Performance Polynomial and Number Theoretic Helpers ---
# Using Numba for JIT compilation and NumPy for efficient array operations.

import numba
import numpy as np
import random

# Enable Numba's Just-In-Time compilation for massive speedup
njit = numba.njit


@njit
def mod_pow_numba(base, exp, mod):
  """A Numba-jitted implementation of modular exponentiation for integers."""
  res = 1
  base %= mod
  while exp > 0:
    if exp % 2 == 1:
      res = (res * base) % mod
    exp //= 2
    base = (base * base) % mod
  return res


@njit
def factorize_numba(n):
  """Finds the unique prime factors of an integer n (Numba-compatible)."""
  factors = []
  d = 2
  temp = n
  while d * d <= temp:
    if temp % d == 0:
      is_in_factors = False
      for f in factors:
        if f == d:
          is_in_factors = True
          break
      if not is_in_factors:
        factors.append(d)
      while temp % d == 0:
        temp //= d
    d += 1
  if temp > 1:
    is_in_factors = False
    for f in factors:
      if f == temp:
        is_in_factors = True
        break
    if not is_in_factors:
      factors.append(temp)
  return factors


@njit
def poly_sub_numba(p, poly1, poly2):
  """Subtracts poly2 from poly1 in GF(p) using NumPy arrays."""
  len1 = len(poly1)
  len2 = len(poly2)
  max_len = max(len1, len2)

  result = np.zeros(max_len, dtype=np.int64)
  result[:len1] += poly1
  result[:len2] -= poly2

  result = (result + p) % p  # Ensure positive results before final mod

  # Normalize result to remove leading zeros
  true_deg = -1
  for i in range(max_len - 1, -1, -1):
    if result[i] != 0:
      true_deg = i
      break
  if true_deg == -1:
    return np.array([0], dtype=np.int64)
  return result[: true_deg + 1]


@njit
def poly_rem_numba(p, dividend, divisor):
  """Calculates remainder of polynomial division in GF(p) using NumPy arrays."""
  rem = dividend.copy()
  deg_divisor = -1
  for i in range(len(divisor) - 1, -1, -1):
    if divisor[i] != 0:
      deg_divisor = i
      break
  if deg_divisor == -1:
    return np.array([0], dtype=np.int64)

  inv = mod_pow_numba(divisor[deg_divisor], p - 2, p)

  for i in range(len(rem) - 1, deg_divisor - 1, -1):
    if rem[i] == 0:
      continue
    factor = (rem[i] * inv) % p
    for j in range(deg_divisor + 1):
      rem[i - deg_divisor + j] = (
          rem[i - deg_divisor + j] - divisor[j] * factor
      ) % p

  true_deg = -1
  for i in range(len(rem) - 1, -1, -1):
    if rem[i] != 0:
      true_deg = i
      break
  if true_deg == -1:
    return np.array([0], dtype=np.int64)
  return rem[: true_deg + 1]


@njit
def poly_mul_numba(p, poly1, poly2, mod_poly):
  """Multiplies two polynomials in GF(p) and reduces by mod_poly."""
  if (len(poly1) == 1 and poly1[0] == 0) or (len(poly2) == 1 and poly2[0] == 0):
    return np.array([0], dtype=np.int64)

  unreduced = np.zeros(len(poly1) + len(poly2) - 1, dtype=np.int64)
  for i1 in range(len(poly1)):
    for i2 in range(len(poly2)):
      unreduced[i1 + i2] = (unreduced[i1 + i2] + poly1[i1] * poly2[i2]) % p

  return poly_rem_numba(p, unreduced, mod_poly)


@njit
def poly_power_numba(p, base, exp, mod_poly):
  """Calculates base^exp for polynomials using exponentiation by squaring."""
  result = np.array([1], dtype=np.int64)
  current_base = base.copy()
  while exp > 0:
    if exp % 2 == 1:
      result = poly_mul_numba(p, result, current_base, mod_poly)
    current_base = poly_mul_numba(p, current_base, current_base, mod_poly)
    exp //= 2
  return result


@njit
def poly_gcd_numba(p, poly1, poly2):
  """Calculates the GCD of two polynomials over GF(p)."""
  a, b = poly1.copy(), poly2.copy()
  while not (len(b) == 1 and b[0] == 0):
    a, b = b, poly_rem_numba(p, a, b)

  if len(a) == 1 and a[0] == 0:
    return a

  lead_coeff = a[-1]
  inv = mod_pow_numba(lead_coeff, p - 2, p)
  return (a * inv) % p


@njit
def is_primitive_numba(p, f):
  """Tests if a polynomial f is primitive over GF(p)."""
  deg = len(f) - 1
  if deg <= 0:
    return False

  x_poly = np.array([0, 1], dtype=np.int64)
  if not np.array_equal(poly_power_numba(p, x_poly, pow(p, deg), f), x_poly):
    return False

  deg_factors = factorize_numba(deg)
  for q in deg_factors:
    k = deg // q
    exp = pow(p, k)
    power_poly = poly_power_numba(p, x_poly, exp, f)
    # BUG FIX: Use the new robust polynomial subtraction function
    h = poly_sub_numba(p, power_poly, x_poly)
    if not np.array_equal(
        poly_gcd_numba(p, f, h), np.array([1], dtype=np.int64)
    ):
      return False

  order = pow(p, deg) - 1
  prime_factors_of_order = factorize_numba(order)

  for q_factor in prime_factors_of_order:
    exponent = order // q_factor
    if np.array_equal(
        poly_power_numba(p, x_poly, exponent, f), np.array([1], dtype=np.int64)
    ):
      return False

  return True


def find_primitive_polynomials_fast(p, degree, max_tries=1000000):
  """A generator that finds primitive polynomials via random search."""
  if degree == 0:
    return

  # print(f"[DEBUG] Starting random search for primitive polynomial (max {max_tries} tries)...")
  for i in range(max_tries):
    coeffs = np.random.randint(0, p, size=degree, dtype=np.int64)
    poly = np.concatenate((coeffs, np.array([1], dtype=np.int64)))

    if is_primitive_numba(p, poly):
      # print(f"\n[DEBUG] Success! Found a primitive polynomial after {i+1} tries.")
      yield poly
      return


# The rest of your code, including `generate_singer_sets` and the non-Numba helpers,
# can remain as it was in the previous version, as the change is isolated to the
# Numba-jitted section.

# --- Main Singer Set Generation Logic ---


def generate_singer_sets(p, n):
  """Generates Singer difference sets with a fast, dynamic search."""
  # Cast to standard Python int for parameter validation and display.
  # The Numba functions will handle NumPy types internally.
  p = int(p)
  n = int(n)

  # print("------------------------------------------------------")
  # print(f"Starting generation for p={p}, n={n} (Fast Dynamic Search)")
  # print("------------------------------------------------------\n")

  m = p**n
  q = m**2 + m + 1

  # print(f"[DEBUG] Parameters calculated:")
  # print(f"  m = p^n = {m}")
  # print(f"  q = m^2 + m + 1 = {q}")
  # print(f"  The difference set will have m + 1 = {m+1} elements.\n")

  degree = 3 * n

  poly_generator = find_primitive_polynomials_fast(p, degree)
  modulus_poly_np = next(poly_generator, None)

  if modulus_poly_np is None:
    print(
        f'[DEBUG] ERROR: Could not find a suitable primitive polynomial after'
        f' max tries.'
    )
    return []

  # Convert back to Python lists for the original logic, which is not performance-critical
  modulus_poly = [int(c) for c in modulus_poly_np]
  # print(f"[DEBUG] Found and using modulus polynomial: {modulus_poly}\n")

  # This part of the logic is not the bottleneck, so we can convert back to lists
  # and reuse the previously verified pure Python functions.
  # (Re-defining them here for completeness)

  def poly_add(p, poly1, poly2):
    len1, len2 = len(poly1), len(poly2)
    result = [0] * max(len1, len2)
    for i in range(len(result)):
      c1 = poly1[i] if i < len1 else 0
      c2 = poly2[i] if i < len2 else 0
      result[i] = (c1 + c2) % p
    while len(result) > 1 and result[-1] == 0:
      result.pop()
    return result

  def poly_rem(p, dividend, divisor):
    rem = list(dividend)
    deg_divisor = len(divisor) - 1
    if deg_divisor < 0:
      return [0]
    inv = pow(divisor[-1], p - 2, p)
    while len(rem) - 1 >= deg_divisor:
      lead_coeff = rem.pop()
      if lead_coeff == 0:
        continue
      factor = (lead_coeff * inv) % p
      for i in range(deg_divisor):
        rem[len(rem) - deg_divisor + i] = (
            rem[len(rem) - deg_divisor + i] - divisor[i] * factor
        ) % p
    while len(rem) > 1 and rem[-1] == 0:
      rem.pop()
    if not rem:
      return [0]
    return rem

  def poly_mul(p, poly1, poly2, mod_poly):
    if poly1 == [0] or poly2 == [0]:
      return [0]
    unreduced = [0] * (len(poly1) + len(poly2) - 1)
    for i1, c1 in enumerate(poly1):
      for i2, c2 in enumerate(poly2):
        unreduced[i1 + i2] = (unreduced[i1 + i2] + c1 * c2) % p
    return poly_rem(p, unreduced, mod_poly)

  def poly_power(p, base, exp, mod_poly):
    result = [1]
    current_base = list(base)
    while exp > 0:
      if exp % 2 == 1:
        result = poly_mul(p, result, current_base, mod_poly)
      current_base = poly_mul(p, current_base, current_base, mod_poly)
      exp //= 2
    return result

  lambda_poly = [0, 1]
  subfield_gen_poly = poly_power(p, lambda_poly, q, modulus_poly)

  subfield_elements = [[0]]
  current_sf_power = [1]
  for _ in range(p**n - 1):
    subfield_elements.append(current_sf_power)
    current_sf_power = poly_mul(
        p, current_sf_power, subfield_gen_poly, modulus_poly
    )

  line_elements = {}
  for c1 in subfield_elements:
    term1 = poly_mul(p, c1, lambda_poly, modulus_poly)
    for c0 in subfield_elements:
      if c1 == [0] and c0 == [0]:
        continue
      line_element = poly_add(p, term1, c0)
      line_elements[tuple(line_element)] = True

  base_difference_set = []
  current_power = [1]
  for k in range(q):
    if tuple(current_power) in line_elements:
      base_difference_set.append(k)
    current_power = poly_mul(p, current_power, lambda_poly, modulus_poly)

  # print(f"\n[DEBUG] Found the base difference set: {base_difference_set}")
  if len(base_difference_set) != m + 1:
    print(
        f'[DEBUG] Error: Generated set size {len(base_difference_set)} !='
        f' {m+1}. Aborting.'
    )
    return []

  # print("\n[DEBUG] Generating all distinct difference sets based on multipliers.")
  coprime_t = [t for t in range(1, q) if math.gcd(t, q) == 1]
  distinct_sets = []
  used_multipliers = set()
  for t in coprime_t[:100]:
    if t in used_multipliers:
      continue
    new_set = sorted([(t * d) % q for d in base_difference_set])
    distinct_sets.append(new_set)
    for i in range(3 * n):
      used_multipliers.add((t * (p**i)) % q)

  # print("\n------------------------------------------------------")
  # print("Generation Complete.")
  # print("------------------------------------------------------")
  return distinct_sets


# --- Leech-like Combination and Empirical n Calculation ---
def combine_A_B(A_set, B_set, m_modulus_for_B):
  """Combines A and B using L = {a*m + b} and finds empirical_n.

  m_modulus_for_B is the 'm' used when constructing B.
  """
  # print(f"\nCombining A={A_set} and B={B_set} with m(N)={m_modulus_for_B}")
  L_set = set()
  for a_val in A_set:
    for b_val in B_set:
      L_set.add(a_val * m_modulus_for_B + b_val)

  if not L_set:
    return []
  sorted_L = sorted(list(L_set))
  return sorted_L


def search_for_best_basis() -> List[int]:
  variable_name = f'best_B_prev'
  current_B_list: List[int]
  if variable_name in globals():
    current_B_list = globals()[variable_name]
  else:
    current_B_list = [0]

  # Ensure 0 is in the set for the greedy completion strategy.
  if 0 not in current_B_list:
    current_B_list.append(0)
  current_B_list = sorted(list(set(current_B_list)))

  best_B_list = list(current_B_list)  # Work with copies
  best_score = get_score(best_B_list)

  start_time = time.time()
  eval_count = 0

  run_duration = 20000
  while time.time() - start_time < run_duration:
    TARGET_A_COVERAGE = np.random.randint(5) + 3
    MAX_ELEMENTS_A = TARGET_A_COVERAGE // 2 + 3
    MAX_VAL_A = TARGET_A_COVERAGE * 3 + np.random.randint(10)

    p_b = np.random.choice([2, 3, 5, 7, 11, 13])  # must be primes
    n_b = np.random.randint(3) + 1
    M_MODULUS_B = (p_b**n_b) ** 2 + p_b**n_b + 1  # don't change this line!

    A = greedy_difference_basis(TARGET_A_COVERAGE, MAX_ELEMENTS_A, MAX_VAL_A)
    A = [0, 1, 4, 6]
    singer_sets = generate_singer_sets(p_b, n_b)
    # print(singer_sets)
    B = random.choice(singer_sets)
    # print(B)

    combined_set = combine_A_B(A, B, M_MODULUS_B)
    current_score = get_score(combined_set)
    eval_count += 1

    # print(f'Current score: {current_score:.4f}, params: {TARGET_A_COVERAGE}, {MAX_ELEMENTS_A}, {MAX_VAL_A}, {M_MODULUS_B}, {MAX_ELEMENTS_B}')

    if current_score > best_score:
      best_score = current_score
      best_B_list = combined_set.copy()
      print(
          f'New best score: {best_score:.4f}, B_size(cand)={len(best_B_list)},'
          f' B_cand={best_B_list[:10]}...'
      )
    # print(f"Best score so far: {best_score}")
  # print(f'Final k={k_target}, Best score: {best_score:.4f}, B_size(cand)={len(best_B_list)}, Evals: {eval_count}')
  return best_B_list


In [ ]:
#@title Final evolved program

import itertools
import logging
import time
from scipy import integrate
import numpy as np
from scipy import optimize
import warnings
import random
import re
from collections.abc import Callable, Mapping, Tuple, Set
from typing import Any, List, Tuple
import scipy.linalg as la
import collections
import copy
import math
import numba

njit = numba.njit

def greedy_difference_basis(target_coverage_A, max_elements_A, max_val_A):
  """Greedily constructs a difference basis A trying to cover 1 to target_coverage_A.

  Stops if target_coverage_A is met, max_elements_A is reached, or no good next
  element is found.
  """
  A = {0}  # Start with 0 to make differences easier
  differences_A = {0}  # To store |a_i - a_j|

  current_max_covered_A = 0

  # print(f"Greedy A: Target coverage {target_coverage_A}, max elements {max_elements_A}, max val {max_val_A}")

  for _ in range(max_elements_A - 1):  # We already have one element (0)
    best_next_a = -1
    best_new_coverage_extension = (
        current_max_covered_A  # How far does it extend contiguous coverage
    )

    # Try candidate next elements
    # Candidates could be random or sequential
    for candidate_a in range(1, max_val_A + 1):
      if candidate_a in A:
        continue

      # Calculate new differences if candidate_a is added
      temp_new_diffs = set()
      for existing_a in A:
        temp_new_diffs.add(abs(candidate_a - existing_a))

      # Check how far contiguous coverage extends
      temp_all_diffs = differences_A.union(temp_new_diffs)

      temp_max_contiguously_covered = 0
      for i in range(
          1, len(temp_all_diffs) + target_coverage_A + 2
      ):  # Check a bit beyond
        if i not in temp_all_diffs:
          temp_max_contiguously_covered = i - 1
          break
      else:  # All checked were covered
        temp_max_contiguously_covered = (
            max(temp_all_diffs) if temp_all_diffs else 0
        )

      if temp_max_contiguously_covered > best_new_coverage_extension:
        best_new_coverage_extension = temp_max_contiguously_covered
        best_next_a = candidate_a

      if temp_max_contiguously_covered == best_new_coverage_extension:
        if np.random.rand() < 0.5:  # Randomly break ties
          best_next_a = candidate_a

    if best_next_a != -1:
      # print(f"  Greedy A: Adding {best_next_a}. New contiguous coverage extends to {best_new_coverage_extension}")
      A.add(best_next_a)
      new_diffs_with_best_a = set()
      for existing_a in A - {
          best_next_a
      }:  # Differences with elements already in A before adding best_next_a
        new_diffs_with_best_a.add(abs(best_next_a - existing_a))
      differences_A.update(new_diffs_with_best_a)
      current_max_covered_A = best_new_coverage_extension
      if current_max_covered_A >= target_coverage_A:
        # print(f"  Greedy A: Reached target coverage {target_coverage_A}.")
        break
    else:
      # print("  Greedy A: No element found to extend contiguous coverage significantly.")
      break

  # print(f"Greedy A: Final set A = {sorted(list(A))}, covers 1 to {current_max_covered_A}")
  return sorted(list(A))


# --- Greedy Algorithm for "Covering" Difference Set B (mod m) ---
def greedy_covering_set_B(m_modulus, max_elements_B):
  """Greedily constructs a set B such that {b_i - b_j mod m} covers all non-zero residues mod m."""
  B = {0}  # Start with 0
  covered_residues_B = {0}  # Stores { (b_i - b_j) mod m }

  # print(f"Greedy B: Modulus m={m_modulus}, max elements {max_elements_B}")

  for _ in range(max_elements_B - 1):  # We already have one element (0)
    if len(covered_residues_B) == m_modulus:  # All residues (0 to m-1) covered
      # print("  Greedy B: All residues modulo m are covered.")
      break

    best_next_b = -1
    max_newly_covered_count = -1

    # Try candidate next elements (0 to m-1)
    # To make it somewhat random but systematic, shuffle candidates
    candidates_b = list(range(m_modulus))
    random.shuffle(candidates_b)

    for candidate_b in candidates_b:
      if candidate_b in B:
        continue

      # Calculate new residues if candidate_b is added
      temp_newly_covered_by_candidate = set()
      for existing_b in B:
        temp_newly_covered_by_candidate.add(
            (candidate_b - existing_b + m_modulus) % m_modulus
        )
        temp_newly_covered_by_candidate.add(
            (existing_b - candidate_b + m_modulus) % m_modulus
        )

      # Count how many *new* residues this candidate_b would cover
      # (residues not already in covered_residues_B)
      count_of_actually_new = len(
          temp_newly_covered_by_candidate - covered_residues_B
      )

      if count_of_actually_new > max_newly_covered_count:
        max_newly_covered_count = count_of_actually_new
        best_next_b = candidate_b

      if count_of_actually_new == max_newly_covered_count:
        if np.random.rand() < 0.5:  # Randomly break ties
          best_next_b = candidate_b

    if (
        best_next_b != -1 and max_newly_covered_count > 0
    ):  # Add if it covers at least one new residue
      # print(f"  Greedy B: Adding {best_next_b}. Covers {max_newly_covered_count} new residue(s).")
      B.add(best_next_b)
      # Update all covered residues with the new element
      new_diffs_with_best_b = set()
      for existing_b in B - {best_next_b}:
        new_diffs_with_best_b.add(
            (best_next_b - existing_b + m_modulus) % m_modulus
        )
        new_diffs_with_best_b.add(
            (existing_b - best_next_b + m_modulus) % m_modulus
        )
      new_diffs_with_best_b.add(0)  # Difference with itself
      covered_residues_B.update(new_diffs_with_best_b)
    else:
      # print("  Greedy B: No element found to cover significantly more new residues, or all covered.")
      break

  all_target_residues = set(range(m_modulus))
  uncovered = all_target_residues - covered_residues_B
  if uncovered:
    print(
        '  Greedy B: Warning! Failed to cover all residues. Uncovered:'
        f' {uncovered}'
    )
  else:
    # print(f"  Greedy B: Successfully covered all residues 0 to {m_modulus-1}.")
    pass

  # print(f"Greedy B: Final set B = {sorted(list(B))}, covers {len(covered_residues_B)}/{m_modulus} residues mod {m_modulus}")
  return sorted(list(B))


# --- Polynomial and Number Theoretic Helpers ---
njit = numba.njit

# --- High-Performance Polynomial and Number Theoretic Helpers ---
# Using Numba for JIT compilation and NumPy for efficient array operations.

import numba
import numpy as np
import random

# Enable Numba's Just-In-Time compilation for massive speedup
njit = numba.njit


@njit
def mod_pow_numba(base, exp, mod):
  """A Numba-jitted implementation of modular exponentiation for integers."""
  res = 1
  base %= mod
  while exp > 0:
    if exp % 2 == 1:
      res = (res * base) % mod
    exp //= 2
    base = (base * base) % mod
  return res


@njit
def factorize_numba(n):
  """Finds the unique prime factors of an integer n (Numba-compatible)."""
  factors = []
  d = 2
  temp = n
  while d * d <= temp:
    if temp % d == 0:
      is_in_factors = False
      for f in factors:
        if f == d:
          is_in_factors = True
          break
      if not is_in_factors:
        factors.append(d)
      while temp % d == 0:
        temp //= d
    d += 1
  if temp > 1:
    is_in_factors = False
    for f in factors:
      if f == temp:
        is_in_factors = True
        break
    if not is_in_factors:
      factors.append(temp)
  return factors


@njit
def poly_sub_numba(p, poly1, poly2):
  """Subtracts poly2 from poly1 in GF(p) using NumPy arrays."""
  len1 = len(poly1)
  len2 = len(poly2)
  max_len = max(len1, len2)

  result = np.zeros(max_len, dtype=np.int64)
  result[:len1] += poly1
  result[:len2] -= poly2

  result = (result + p) % p  # Ensure positive results before final mod

  # Normalize result to remove leading zeros
  true_deg = -1
  for i in range(max_len - 1, -1, -1):
    if result[i] != 0:
      true_deg = i
      break
  if true_deg == -1:
    return np.array([0], dtype=np.int64)
  return result[: true_deg + 1]


@njit
def poly_rem_numba(p, dividend, divisor):
  """Calculates remainder of polynomial division in GF(p) using NumPy arrays."""
  rem = dividend.copy()
  deg_divisor = -1
  for i in range(len(divisor) - 1, -1, -1):
    if divisor[i] != 0:
      deg_divisor = i
      break
  if deg_divisor == -1:
    return np.array([0], dtype=np.int64)

  inv = mod_pow_numba(divisor[deg_divisor], p - 2, p)

  for i in range(len(rem) - 1, deg_divisor - 1, -1):
    if rem[i] == 0:
      continue
    factor = (rem[i] * inv) % p
    for j in range(deg_divisor + 1):
      rem[i - deg_divisor + j] = (
          rem[i - deg_divisor + j] - divisor[j] * factor
      ) % p

  true_deg = -1
  for i in range(len(rem) - 1, -1, -1):
    if rem[i] != 0:
      true_deg = i
      break
  if true_deg == -1:
    return np.array([0], dtype=np.int64)
  return rem[: true_deg + 1]


@njit
def poly_mul_numba(p, poly1, poly2, mod_poly):
  """Multiplies two polynomials in GF(p) and reduces by mod_poly."""
  if (len(poly1) == 1 and poly1[0] == 0) or (len(poly2) == 1 and poly2[0] == 0):
    return np.array([0], dtype=np.int64)

  unreduced = np.zeros(len(poly1) + len(poly2) - 1, dtype=np.int64)
  for i1 in range(len(poly1)):
    for i2 in range(len(poly2)):
      unreduced[i1 + i2] = (unreduced[i1 + i2] + poly1[i1] * poly2[i2]) % p

  return poly_rem_numba(p, unreduced, mod_poly)


@njit
def poly_power_numba(p, base, exp, mod_poly):
  """Calculates base^exp for polynomials using exponentiation by squaring."""
  result = np.array([1], dtype=np.int64)
  current_base = base.copy()
  while exp > 0:
    if exp % 2 == 1:
      result = poly_mul_numba(p, result, current_base, mod_poly)
    current_base = poly_mul_numba(p, current_base, current_base, mod_poly)
    exp //= 2
  return result


@njit
def poly_gcd_numba(p, poly1, poly2):
  """Calculates the GCD of two polynomials over GF(p)."""
  a, b = poly1.copy(), poly2.copy()
  while not (len(b) == 1 and b[0] == 0):
    a, b = b, poly_rem_numba(p, a, b)

  if len(a) == 1 and a[0] == 0:
    return a

  lead_coeff = a[-1]
  inv = mod_pow_numba(lead_coeff, p - 2, p)
  return (a * inv) % p


@njit
def is_primitive_numba(p, f):
  """Tests if a polynomial f is primitive over GF(p)."""
  deg = len(f) - 1
  if deg <= 0:
    return False

  x_poly = np.array([0, 1], dtype=np.int64)
  if not np.array_equal(poly_power_numba(p, x_poly, pow(p, deg), f), x_poly):
    return False

  deg_factors = factorize_numba(deg)
  for q in deg_factors:
    k = deg // q
    exp = pow(p, k)
    power_poly = poly_power_numba(p, x_poly, exp, f)
    # BUG FIX: Use the new robust polynomial subtraction function
    h = poly_sub_numba(p, power_poly, x_poly)
    if not np.array_equal(
        poly_gcd_numba(p, f, h), np.array([1], dtype=np.int64)
    ):
      return False

  order = pow(p, deg) - 1
  prime_factors_of_order = factorize_numba(order)

  for q_factor in prime_factors_of_order:
    exponent = order // q_factor
    if np.array_equal(
        poly_power_numba(p, x_poly, exponent, f), np.array([1], dtype=np.int64)
    ):
      return False

  return True


def find_primitive_polynomials_fast(p, degree, max_tries=1000000):
  """A generator that finds primitive polynomials via random search."""
  if degree == 0:
    return

  # print(f"[DEBUG] Starting random search for primitive polynomial (max {max_tries} tries)...")
  for i in range(max_tries):
    coeffs = np.random.randint(0, p, size=degree, dtype=np.int64)
    poly = np.concatenate((coeffs, np.array([1], dtype=np.int64)))

    if is_primitive_numba(p, poly):
      # print(f"\n[DEBUG] Success! Found a primitive polynomial after {i+1} tries.")
      yield poly
      return


# The rest of your code, including `generate_singer_sets` and the non-Numba helpers,
# can remain as it was in the previous version, as the change is isolated to the
# Numba-jitted section.

# --- Main Singer Set Generation Logic ---


def generate_singer_sets(p, n):
  """Generates Singer difference sets with a fast, dynamic search."""
  # Cast to standard Python int for parameter validation and display.
  # The Numba functions will handle NumPy types internally.
  p = int(p)
  n = int(n)

  # print("------------------------------------------------------")
  # print(f"Starting generation for p={p}, n={n} (Fast Dynamic Search)")
  # print("------------------------------------------------------\n")

  m = p**n
  q = m**2 + m + 1

  # print(f"[DEBUG] Parameters calculated:")
  # print(f"  m = p^n = {m}")
  # print(f"  q = m^2 + m + 1 = {q}")
  # print(f"  The difference set will have m + 1 = {m+1} elements.\n")

  degree = 3 * n

  poly_generator = find_primitive_polynomials_fast(p, degree)
  modulus_poly_np = next(poly_generator, None)

  if modulus_poly_np is None:
    print(
        f'[DEBUG] ERROR: Could not find a suitable primitive polynomial after'
        f' max tries.'
    )
    return []

  # Convert back to Python lists for the original logic, which is not performance-critical
  modulus_poly = [int(c) for c in modulus_poly_np]
  # print(f"[DEBUG] Found and using modulus polynomial: {modulus_poly}\n")

  # This part of the logic is not the bottleneck, so we can convert back to lists
  # and reuse the previously verified pure Python functions.
  # (Re-defining them here for completeness)

  def poly_add(p, poly1, poly2):
    len1, len2 = len(poly1), len(poly2)
    result = [0] * max(len1, len2)
    for i in range(len(result)):
      c1 = poly1[i] if i < len1 else 0
      c2 = poly2[i] if i < len2 else 0
      result[i] = (c1 + c2) % p
    while len(result) > 1 and result[-1] == 0:
      result.pop()
    return result

  def poly_rem(p, dividend, divisor):
    rem = list(dividend)
    deg_divisor = len(divisor) - 1
    if deg_divisor < 0:
      return [0]
    inv = pow(divisor[-1], p - 2, p)
    while len(rem) - 1 >= deg_divisor:
      lead_coeff = rem.pop()
      if lead_coeff == 0:
        continue
      factor = (lead_coeff * inv) % p
      for i in range(deg_divisor):
        rem[len(rem) - deg_divisor + i] = (
            rem[len(rem) - deg_divisor + i] - divisor[i] * factor
        ) % p
    while len(rem) > 1 and rem[-1] == 0:
      rem.pop()
    if not rem:
      return [0]
    return rem

  def poly_mul(p, poly1, poly2, mod_poly):
    if poly1 == [0] or poly2 == [0]:
      return [0]
    unreduced = [0] * (len(poly1) + len(poly2) - 1)
    for i1, c1 in enumerate(poly1):
      for i2, c2 in enumerate(poly2):
        unreduced[i1 + i2] = (unreduced[i1 + i2] + c1 * c2) % p
    return poly_rem(p, unreduced, mod_poly)

  def poly_power(p, base, exp, mod_poly):
    result = [1]
    current_base = list(base)
    while exp > 0:
      if exp % 2 == 1:
        result = poly_mul(p, result, current_base, mod_poly)
      current_base = poly_mul(p, current_base, current_base, mod_poly)
      exp //= 2
    return result

  lambda_poly = [0, 1]
  subfield_gen_poly = poly_power(p, lambda_poly, q, modulus_poly)

  subfield_elements = [[0]]
  current_sf_power = [1]
  for _ in range(p**n - 1):
    subfield_elements.append(current_sf_power)
    current_sf_power = poly_mul(
        p, current_sf_power, subfield_gen_poly, modulus_poly
    )

  line_elements = {}
  for c1 in subfield_elements:
    term1 = poly_mul(p, c1, lambda_poly, modulus_poly)
    for c0 in subfield_elements:
      if c1 == [0] and c0 == [0]:
        continue
      line_element = poly_add(p, term1, c0)
      line_elements[tuple(line_element)] = True

  base_difference_set = []
  current_power = [1]
  for k in range(q):
    if tuple(current_power) in line_elements:
      base_difference_set.append(k)
    current_power = poly_mul(p, current_power, lambda_poly, modulus_poly)

  # print(f"\n[DEBUG] Found the base difference set: {base_difference_set}")
  if len(base_difference_set) != m + 1:
    print(
        f'[DEBUG] Error: Generated set size {len(base_difference_set)} !='
        f' {m+1}. Aborting.'
    )
    return []

  # print("\n[DEBUG] Generating all distinct difference sets based on multipliers.")
  coprime_t = [t for t in range(1, q) if math.gcd(t, q) == 1]
  distinct_sets = []
  used_multipliers = set()
  for t in coprime_t[:100]:
    if t in used_multipliers:
      continue
    new_set = sorted([(t * d) % q for d in base_difference_set])
    distinct_sets.append(new_set)
    for i in range(3 * n):
      used_multipliers.add((t * (p**i)) % q)

  # print("\n------------------------------------------------------")
  # print("Generation Complete.")
  # print("------------------------------------------------------")
  return distinct_sets


# --- Leech-like Combination and Empirical n Calculation ---
def combine_A_B(A_set, B_set, m_modulus_for_B):
  """Combines A and B using L = {a*m + b} and finds empirical_n.

  m_modulus_for_B is the 'm' used when constructing B.
  """
  # print(f"\nCombining A={A_set} and B={B_set} with m(N)={m_modulus_for_B}")
  L_set = set()
  for a_val in A_set:
    for b_val in B_set:
      L_set.add(a_val * m_modulus_for_B + b_val)

  if not L_set:
    return []
  sorted_L = sorted(list(L_set))
  return sorted_L


def search_for_best_basis() -> List[int]:
  variable_name = f'best_B_prev'
  current_B_list: List[int]
  if variable_name in globals():
    current_B_list = globals()[variable_name]
  else:
    current_B_list = [0]

  # Ensure 0 is in the set for the greedy completion strategy.
  if 0 not in current_B_list:
    current_B_list.append(0)
  current_B_list = sorted(list(set(current_B_list)))

  best_B_list = list(current_B_list)  # Work with copies
  best_score = get_score(best_B_list)

  # Ensure 0 is in the set for the greedy completion strategy.
  # This part is now handled if current_B_list is loaded.
  # if 0 not in current_B_list:
  # current_B_list.append(0)
  # current_B_list = sorted(list(set(current_B_list)))

  # Properly initialize from best_B_prev or a default
  if variable_name in globals():
    current_B_list = globals()[variable_name]
    if not current_B_list: # If best_B_prev was an empty list
        current_B_list = [0]
    elif 0 not in current_B_list: # Ensure 0 is present
        current_B_list.append(0)
        current_B_list = sorted(list(set(current_B_list)))
  else:
    current_B_list = [0]


  best_B_list = list(current_B_list)
  best_score = get_score(best_B_list) # Score the initial list

  start_time = time.time()
  eval_count = 0
  run_duration = 20000

  # Constants for Singer set parameter choices
  SINGER_PARAM_OPTIONS = [
    (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), # m = 2, 4, 8, 16, 32, 64
    (3, 1), (3, 2), (3, 3), (3, 4),               # m = 3, 9, 27, 81
    (5, 1), (5, 2), (5, 3),                       # m = 5, 25, 125
    (7, 1), (7, 2),                               # m = 7, 49
    # Primes up to 97
    (11, 1), (13, 1), (17, 1), (19, 1), (23, 1), (29, 1), (31, 1),
    (37, 1), (41, 1), (43, 1), (47, 1), (53, 1), (59, 1), (61, 1),
    (67, 1), (71, 1), (73, 1), (79, 1), (83, 1), (89, 1), (97, 1)
  ]

  while time.time() - start_time < run_duration:
    # With a small probability, try to mutate the best set found so far.
    # Otherwise, generate a new set from scratch using the construction.
    if np.random.rand() < 0.9 or len(best_B_list) < 2: # 90% chance for construction
      # --- Generate Set A (Construction Branch) ---
      A_set_generated: List[int]
      if np.random.rand() < 0.7: # 70% chance to use the best known small A
          A_set_generated = [0, 1, 4, 6] # k_A = 6, |A|=4, score ~2.66
      else: # 30% chance to use greedy A
          target_A_cov = np.random.randint(2, 9)
          max_elem_A = np.random.randint(3, 7)
          min_max_val_A = target_A_cov
          max_val_A = np.random.randint(max(min_max_val_A, 5), max(min_max_val_A, 10) + 25)
          A_set_generated = greedy_difference_basis(target_A_cov, max_elem_A, max_val_A)

      if len(A_set_generated) < 2:
          continue

      # --- Generate Set B and M_MODULUS_B ---
      B_set_generated: List[int]
      M_MODULUS_B: int

      use_singer = np.random.rand() < 0.8 # 80% chance for Singer set
      if use_singer:
          (p_b, n_b) = random.choice(SINGER_PARAM_OPTIONS)
          m_val = p_b ** n_b
          M_MODULUS_B = m_val**2 + m_val + 1

          try:
              singer_sets = generate_singer_sets(p_b, n_b)
          except Exception: # Catch any error from singer set gen
              continue

          if not singer_sets:
              continue
          B_set_generated = random.choice(singer_sets)
      else: # 20% chance for Greedy B
          M_MODULUS_B = np.random.randint(20, 200)
          min_b_elems = max(3, int(M_MODULUS_B**0.5))
          max_b_elems = max(min_b_elems + 1, int(M_MODULUS_B * 0.7) + 2)
          num_elem_B = np.random.randint(min_b_elems, max_b_elems + 1)
          B_set_generated = greedy_covering_set_B(M_MODULUS_B, num_elem_B)

      if len(B_set_generated) < 2:
          continue

      # --- Combine and Score ---
      combined_set = combine_A_B(A_set_generated, B_set_generated, M_MODULUS_B)
      if not combined_set:
          continue

      current_score = get_score(combined_set)
      eval_count += 1

      if current_score > best_score:
          best_score = current_score
          best_B_list = combined_set # combined_set is already a sorted list
          logging.info(
              f'New best score (construction): {best_score:.4f}, |L|={len(best_B_list)}, A_size={len(A_set_generated)}, B_size={len(B_set_generated)}, M={M_MODULUS_B}'
          )
    else: # 10% chance to mutate
        # --- Mutation Branch ---
        # 1. Find k for the current best set
        differences = set()
        for i in range(len(best_B_list)):
            for j in range(i + 1, len(best_B_list)):
                differences.add(best_B_list[j] - best_B_list[i])

        k = 0
        if differences:
            max_d = max(differences)
            for i in range(1, max_d + 2):
                if i not in differences:
                    k = i - 1
                    break
            else:
                k = max_d

        # 2. Generate candidates to add to cover k+1
        target_to_cover = k + 1
        candidates = {b + target_to_cover for b in best_B_list}
        candidates.add(best_B_list[-1] + 1) # Simple heuristic: extend by 1

        # 3. Test candidates and update if a better set is found
        best_mutant_set = None
        best_mutant_score = best_score

        # To avoid being too slow, sample a limited number of candidates
        cand_list = list(candidates)
        if len(cand_list) > 100:
            cand_list = random.sample(cand_list, 100)

        original_set_for_diff = set(best_B_list)
        for cand_x in cand_list:
            if cand_x in best_B_list: continue

            trial_set = sorted(best_B_list + [cand_x])
            score = get_score(trial_set)
            eval_count += 1
            if score > best_mutant_score:
                best_mutant_score = score
                best_mutant_set = trial_set

        if best_mutant_set:
            added_element = list(set(best_mutant_set) - original_set_for_diff)[0]
            best_score = best_mutant_score
            best_B_list = best_mutant_set
            logging.info(
                f'New best score (mutation): {best_score:.4f}, |L|={len(best_B_list)}, added {added_element}'
            )

  return best_B_list

